In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
#sys.path.append('shallow')

# Imports

In [ ]:
#export
from shallow import nb_utils
#from nb_utils import GetAttr

# Code

In [ ]:
#export 
class Callback(nb_utils.GetAttr): _default='learner'

class ParamScheduler(Callback):
    def __init__(self, phase, pname, sched_func):
        self.pname, self.sched_func = pname, sched_func
        setattr(self, phase, self.set_param)
        
    def set_param(self):
        setattr(self.learner, self.pname, self.sched_func(self.n_epochs/self.epochs))
    
class SetupLearnerCB(Callback):
    def before_batch(self):
        xb,yb = to_device(self.batch)
        self.learner.batch = tfm_x(xb),yb

    def before_fit(self): self.model.cuda()

class TrackResults(Callback):
    def before_epoch(self): self.accs,self.losses,self.ns = [],[],[]
        
    def after_epoch(self):
        n = sum(self.ns)
        print(self.epoch, self.model.training,
              sum(self.losses).item()/n, sum(self.accs).item()/n)
        
    def after_batch(self):
        xb,yb = self.batch
        acc = (self.preds.argmax(dim=1)==yb).float().sum()
        self.accs.append(acc)
        n = len(xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

class LRFinder(Callback):
    def before_fit(self):
        self.losses,self.lrs = [],[]
        self.learner.lr = 1e-6
        
    def before_batch(self):
        if not self.model.training: return
        self.opt.lr *= 1.2

    def after_batch(self):
        if not self.model.training: return
        if self.opt.lr>10 or torch.isnan(self.loss): raise CancelFitException
        self.losses.append(self.loss.item())
        self.lrs.append(self.opt.lr)

In [ ]:
from functools import partial

def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner

@annealer
def sched_lin(start, end, pos): return start + pos*(end-start)

In [ ]:
f = sched_lin(0,10)

In [ ]:
f(.13)

In [ ]:
p = ParamScheduler('before_epoch', 'lr', f)

In [ ]:
class Learner:
    def __init__(self, cbs):
        self.cbs = cbs
        self.lr = -1
        self.epochs = 100
        self.n_epochs = 13
        for c in self.cbs: c.learner=self
    
    def t(self):
        self('before_epoch')
        
    def tt(self):
        self('bla_bla')
        
    def __call__(self, name):
        for cb in self.cbs: getattr(cb, name, nb_utils.noop())()

In [ ]:
l = Learner([p])

In [ ]:
l.lr

In [ ]:
l.n_epochs = 88
l.t()